In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

# This shows the mean, standard deviation, minimum, and other metrics for our numeric columns.
df.describe()

# Get some data on our boolean column indicating the baby's gender.
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [8]:
# Drop rows with null values from the dataset and shuffle the data.
df = df.dropna()
df = shuffle(df, random_state=2)

# Extract the label column into a separate variable and create a DataFrame with only our features.
# Since is_male is a boolean, we'll convert it to an integer so that all inputs to our model are numeric.
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
757,7.374463,True,23,1,39.0
6544,8.375361,True,19,1,38.0
3190,6.239082,True,28,1,40.0
2064,8.437091,False,32,1,40.0
4066,7.374463,False,17,1,40.0


In [9]:
# Split your data into train and test sets.
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

# We'll be building this model using the tf.keras Sequential model API,
# which lets us define our model as a stack of layers.
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

# Then we'll compile our model so we can train it.
# Here we'll choose the model's optimizer, loss function,
# and metrics we'd like the model to log during training.
# Since this is a regression model (predicting a numerical value),
# we're using mean squared error instead of accuracy as our metric.
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

# See the shape and number of trainable parameters of your model at each layer.
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Use the optional validation_split parameter,
# which will hold a portion of our training data
# to validate the model at each step.
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

# See how the model is performing.
num_examples = 10
predictions = model.predict(x_test[:num_examples])

Train on 6665 samples, validate on 741 samples
Epoch 1/10
6665/6665 [==============================] - 1s 197us/sample - loss: 2.0354 - mean_absolute_error: 1.0400 - mean_squared_error: 2.0354 - val_loss: 1.1952 - val_mean_absolute_error: 0.8583 - val_mean_squared_error: 1.1952
Epoch 2/10
6665/6665 [==============================] - 1s 119us/sample - loss: 1.4552 - mean_absolute_error: 0.9490 - mean_squared_error: 1.4552 - val_loss: 1.5479 - val_mean_absolute_error: 0.9907 - val_mean_squared_error: 1.5479
Epoch 3/10
6665/6665 [==============================] - 1s 111us/sample - loss: 1.3960 - mean_absolute_error: 0.9290 - mean_squared_error: 1.3960 - val_loss: 1.1328 - val_mean_absolute_error: 0.8352 - val_mean_squared_error: 1.1328
Epoch 4/10
6665/6665 [==============================] - 1s 101us/sample - loss: 1.3683 - mean_absolute_error: 0.9234 - mean_squared_error: 1.3683 - val_loss: 1.8199 - val_mean_absolute_error: 1.0848 - val_mean_squared_error: 1.8199
Epoch 5/10
6665/6665 [===